# Protocol Analysis For Privacy

In [12]:
import pandas as pd

df = pd.read_csv('/home/uday/Desktop/cap/pcaps/test_dec.csv', encoding='ISO-8859-1')
# df_2 = pd.read_csv('../data/oct18-21.csv', encoding='ISO-8859-1')
# df_3 = pd.read_csv('../data/oct-21.csv', encoding='ISO-8859-1')

# df = pd.concat([df_1, df_2, df_3], ignore_index=True)

columns_of_interest = ['Protocol','Source', 'Length', 'TTL', 'Src Port', 'Dst Port', 'Tcp Window', 'dns.qry_name']

# Function to get the top 3 most frequent values and their counts for each column and group
def get_top3_values_per_column(group):
    top3 = {}
    for col in columns_of_interest:
        top_values = group[col].value_counts().nlargest(3)  # Get top 3 values and their counts
        top3[col] = top_values
    return pd.Series(top3)

# Group by 'Src Mac' and apply the function
top3_per_device = df.groupby('Src Mac').apply(get_top3_values_per_column)

output_file = "top3_values_per_device_test_dec.txt"

# Prettify the display and write to file
with open(output_file, 'w') as f:
    for mac, data in top3_per_device.iterrows():
        f.write(f"\nDevice (Src Mac): {mac}\n")
        f.write("-" * 40 + "\n")
        for column, values in data.items():
            f.write(f"{column}:\n")
            if isinstance(values, pd.Series):
                for val, count in values.items():
                    f.write(f"   {val}: {count} times\n")
            else:
                f.write(f"   {values}\n")
        f.write("-" * 40 + "\n")

print(f"Output written to {output_file}")

Output written to top3_values_per_device_test_dec.txt


/tmp/ipykernel_5091/482746355.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top3_per_device = df.groupby('Src Mac').apply(get_top3_values_per_column)


In [13]:
import pandas as pd

# Load the data
df = pd.read_csv('/home/uday/Desktop/cap/pcaps/test_dec.csv', encoding='ISO-8859-1', low_memory=False)

columns_of_interest = [
    'Protocol', 
    'Source', 
    'Destination',
    'Length', 
    'TTL', 
    'Src Port', 
    'Dst Port', 
    'Tcp Window', 
    'dns.qry_name'
]

# Group data by 'Src Mac'
grouped = df.groupby('Src Mac')

# Initialize a list to hold the structured data
structured_data = []

# Process each group
for mac, group in grouped:
    row = {'Device': mac}
    # Add total packet count
    row['Total Packets'] = len(group)
    for col in columns_of_interest:
        if col in group.columns:
            # Get the top 3 most frequent values
            top_values = group[col].dropna().value_counts().nlargest(3).index.tolist()
            # Handle numeric conversions
            top_values = [int(v) if isinstance(v, (int, float, pd.Int64Dtype)) else v for v in top_values]
            row[col] = top_values
    structured_data.append(row)

# Convert the structured data to a DataFrame
output_df = pd.DataFrame(structured_data)

# Add a rotated device order column
original_devices = output_df['Device'].tolist()
devices = original_devices[1:] + original_devices[:1]  # Rotate the list
output_df['Device_t'] = devices

# Save the output to a CSV file
output_file = "output_report_csv.csv"
output_df.to_csv(output_file, index=False)

print(f"Formatted output with top 3 values and total packets written to {output_file}")


Formatted output with top 3 values and total packets written to output_report_csv.csv


In [2]:
import pandas as pd

df = pd.DataFrame(pd.read_csv("top3_values_with_total_packets_tot.csv"))

In [3]:
df

,Device,Total Packets,Protocol,Source,Destination,Length,TTL,Src Port,Dst Port,Tcp Window,dns.qry_name
0,MS-NLB-PhysServer-26_c5:00:00:00,777008,"['TCP', 'BitTorrent', 'IMAP']","['59.166.0.1', '59.166.0.5', '59.166.0.2']","['149.171.126.9', '149.171.126.8', '149.171.12...","[68, 1516, 72]",[],"[28763, 55237, 51774]","[6881, 80, 25]","[3526990, 1, 5]",[]
1,PaloAltoNetw_05:9e:1c,986155,"['TCP', 'HTTP', 'BitTorrent']","['149.171.126.9', '149.171.126.8', '149.171.12...","['59.166.0.1', '59.166.0.2', '59.166.0.9']","[1516, 68, 1368]",[],"[6881, 80, 143]","[14667, 31649, 51100]","[3526992, 3, 11]",[]
2,PaloAltoNetw_05:9e:1e,43,['ARP'],['PaloAltoNetw_05:9e:1e'],[],[62],[],[],[],"[1210, 2590, 4098]",[]
3,VMware_a5:13:c8,7,"['BROWSER', 'ARP']","['10.40.182.6', 'VMware_a5:13:c8']",['10.40.182.255'],"[279, 256, 62]",[],[138],[138],"[4283, 4284, 984558]",[]
4,VMware_a5:24:c2,777176,"['TCP', 'DNS', 'Portmap']","['59.166.0.1', '59.166.0.5', '59.166.0.2']","['149.171.126.9', '149.171.126.8', '149.171.12...","[68, 1516, 72]",[],"[28763, 55237, 51774]","[6881, 80, 25]","[3526991, 2, 6]",[]
5,VMware_a5:5a:a8,72,"['DNS', 'ARP', 'ICMP']","['10.40.85.10', 'VMware_a5:5a:a8']","['10.40.198.10', '192.168.241.50']","[96, 83, 62]",[],"[46774, 33645, 47050]",[53],"[1631, 1773, 4131]",[]
6,VMware_a5:77:63,986531,"['TCP', 'DNS', 'Portmap']","['149.171.126.9', '149.171.126.8', '149.171.12...","['59.166.0.1', '59.166.0.2', '59.166.0.9']","[1516, 68, 1368]",[],"[6881, 80, 143]","[14667, 31649, 51100]","[3526989, 4, 12]",[]


In [4]:
source = df['Source']

In [14]:
import random
import ast

In [15]:
for i in source:
    i = ast.literal_eval(i)
    print(i[1])
    break
    

59.166.0.5


In [9]:
import pandas as pd

# Load the data
df = pd.read_csv('/home/uday/Desktop/cap/pcaps/test_dec.csv', encoding='ISO-8859-1', low_memory=False)

columns_of_interest = [
    'Protocol', 
    'Source', 
    'Destination',
    'Length', 
    'TTL', 
    'Src Port', 
    'Dst Port', 
    'Tcp Window', 
    'dns.qry_name'
]

# Group data by 'Src Mac'
grouped = df.groupby('Src Mac')

# Initialize a list to hold the structured data
structured_data = []

# Process each group
for mac, group in grouped:
    row = {'Device': mac}
    # Add total packet count
    row['Total Packets'] = len(group)
    
    # Process Protocol column for top 3 protocols and their counts
    if 'Protocol' in group.columns:
        protocol_counts = group['Protocol'].dropna().value_counts().nlargest(3)  # Top 3 protocols
        row['Top Protocols'] = protocol_counts.index.tolist()  # List of top 3 protocols
        row['Top Protocol Counts'] = protocol_counts.values.tolist()  # Corresponding counts
    
    # Process other columns of interest
    for col in columns_of_interest:
        if col in group.columns and col != 'Protocol':
            # Get the top 3 most frequent values
            top_values = group[col].dropna().value_counts().nlargest(3).index.tolist()
            row[col] = top_values
    structured_data.append(row)

# Convert the structured data to a DataFrame
output_df = pd.DataFrame(structured_data)

# Save the output to a CSV file
output_file = "output_test_dec.csv"
output_df.to_csv(output_file, index=False)

print(f"Formatted output with top 3 protocols written to {output_file}")


Formatted output with top 3 protocols written to output_test_dec.csv


In [11]:
import pandas as pd

# Load the data
df = pd.read_csv('/home/uday/Desktop/cap/pcaps/test_dec.csv', encoding='ISO-8859-1', low_memory=False)

columns_of_interest = [
    'Protocol', 
    'Source', 
    'Destination',
    'Length', 
    'TTL', 
    'Src Port', 
    'Dst Port', 
    'Tcp Window', 
    'dns.qry_name'
]

# Group data by 'Src Mac'
grouped = df.groupby('Src Mac')

# Initialize a list to hold the structured data
structured_data = []

# Process each group
for mac, group in grouped:
    row = {'Device': mac}
    # Add total packet count
    row['Total Packets'] = len(group)
    
    # Process Protocol column for top 3 protocols and their counts
    if 'Protocol' in group.columns:
        protocol_counts = group['Protocol'].dropna().value_counts().nlargest(3)  # Top 3 protocols
        row['Top Protocols'] = protocol_counts.index.tolist()  # List of top 3 protocols
        row['Top Protocol Counts'] = protocol_counts.values.tolist()  # Corresponding counts
    
    # Process other columns of interest
    for col in columns_of_interest:
        if col in group.columns and col != 'Protocol':
            # Get the top 3 most frequent values
            top_values = group[col].dropna().value_counts().nlargest(3).index.tolist()
            # Ensure numeric values are converted to integers
            top_values = [
                int(v) if (isinstance(v, (int, float, str)) and str(v).isdigit()) else v 
                for v in top_values
            ]
            row[col] = top_values
    structured_data.append(row)

# Convert the structured data to a DataFrame
output_df = pd.DataFrame(structured_data)

# Add a reversed (rotated) device order column
original_devices = output_df['Device'].tolist()
reversed_devices = original_devices[1:] + original_devices[:1]  # Rotate the list
output_df['Reversed Device Order'] = reversed_devices

# Save the output to a CSV file
output_file = "output_fixed_int_columns.csv"
output_df.to_csv(output_file, index=False)

print(f"Output with all numeric columns fixed written to {output_file}")


Output with all numeric columns fixed written to output_fixed_int_columns.csv
